# 4. 이미지 프로세싱 기초

## 4.1 관심 영역(ROI)
* ROI (Region of Interest)  # rectangle 형태를 이야기 함!!

In [8]:
# 관심영역에 초록색 네모 그리기
import cv2
import numpy as np

img = cv2.imread('img/jiheon.jpg')
x = 110; y = 100; w = 300; h = 300
roi = img[y:y+h, x:x+w] # numpy는 copy가 아닌 referencing!!
cv2.rectangle(roi, (0,0), (w-1, h-1), (0,255,0)) # BGR 순서
# rectangle(img, start, end, color [, ... ])
cv2.imshow('img', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
# 영역 복사 붙여넣기
import cv2
import numpy as np

img = cv2.imread('img/jiheon.jpg')
x = 320; y = 150; w = 50; h = 50
roi = img[y:y+h, x:x+w] # numpy는 copy가 아닌 referencing!!
cv2.rectangle(roi, (0,0), (w-1, h-1), (0,255,0)) # BGR 순서
# rectangle(img, start, end, color [, ... ])

img[y:y+h, x+w:x+w+w] = roi # roi를 img에 붙여넣기
cv2.imshow('img', img)
cv2.imwrite('img/jiheon_roi.jpg', roi)

cv2.waitKey(0)
cv2.destroyAllWindows()

### 4.1.2 마우스로 관심영역 지정

* 마우스 이벤트
    * cv2.EVENT_LBUTTONDOWN # 누르기
    * cv2.EVENT_LBUTTONUP # 떼기
    * cv2.EVENT_MOUSEMOVE # 움직이기
        * 드래그란, 누르고 유지하고 당기고 떼고!
        * 윈도우는 마우스를 계~속 처리하고 있는거야

In [22]:
# 드래그 영역 네모 그리기
import cv2
import numpy as np

isDragging = False # 눌린 상태 여부
pos = (-1, -1) # x0, y0
w, h = -1, -1 # init


def onMouse(event, x, y, flags, param):
     # global을 쓰면 local이 아닌 함수 바깥의 변수 사용
    global isDragging, pos, w, h, img
    if event == cv2.EVENT_LBUTTONDOWN:
        isDragging = True
        pos = (x,y) # 처음 누른 위치
    elif event == cv2.EVENT_MOUSEMOVE:
        if isDragging:
            img_draw = img.copy() # 보통 처리할 때는 원본이 아닌 카피 후 진행
            cv2.rectangle(img_draw, pos, (x,y), (255,0,0), 2) # 현재 좌표 x,y
            cv2.imshow('img_draw', img_draw) # 움직일때마다 출력
    elif event == cv2.EVENT_LBUTTONUP:
        if isDragging:
            isDragging = False
            w = x - pos[0]
            h = y - pos[1]
            if w > 0 and h > 0:
                img_draw = img.copy()
                cv2.rectangle(img_draw, pos, (x,y), (0,255,0), 2)
                cv2.imshow('img_draw', img_draw)
                roi = img[pos[1]:pos[1]+h, pos[0]:pos[0]+w]
                cv2.imshow('cropped', roi)
                cv2.imwrite('img/cropped.jpg', roi)
            else:
                cv2.imshow('img_draw', img)

                
img = cv2.imread('img/jiheon.jpg')
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)

cv2.waitKey(0)
cv2.destroyAllWindows()

### openCV로 하면, 위 많은 것들을.. 알아서 다 해준다!

ret = cv2.selectROI([win_name,] img [, showCrossHair = True, fromCenter = False])

* win_name: ROI 선택을 진행할 창의 이름, str
* img: ROI 선택을 진행할 이미지, NumPy ndarray
* showCrossHair: 선택 영역 중심에 십자모양 표시 여부
* fromCenter: 마우스 클릭 시작지점을 영역의 중심으로 지정
* ret: 선택한 영역 좌표와 크기(x,y,w,h), 선택을 취소한 경우 모두 0
        
영역 선택 후, 스페이스나 엔터키를 누르면 좌표와 크기값이 반환되고, 'c'키를 누르면 선택이 취소되고, 리턴값이 0이 된다.

In [24]:
# 드래그 영역 네모 그리기
import cv2
import numpy as np
               
img = cv2.imread('img/jiheon.jpg')
x,y,w,h = cv2.selectROI('img', img, False)
if w and h:
    roi = img[y:y+h, x:x+w]
    cv2.imshow('cropped', roi)
    cv2.imwrite('img/cropped2.jpg', roi) # 저장
    
cv2.imshow('img', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
# 드래그 영역 저장해서 클릭한 좌표에 출력하기!

import cv2
import numpy as np
               
img = cv2.imread('img/jiheon.jpg')
x,y,w,h = cv2.selectROI('img', img, False)
img_draw = img.copy() # call by reference 이므로 copy 해서 사용
max_width = len(img_draw[0])
max_height = len(img_draw)


if w and h:
    roi = img[y:y+h, x:x+w]

    
def onMouse(event, x, y, flags, param):
     # global을 쓰면 local이 아닌 함수 바깥의 변수 사용
    global img_draw, roi, w, h, max_width, max_height
    if event == cv2.EVENT_LBUTTONDOWN:
        roi_draw = roi.copy()
        
        if y+h > max_height and x+w > max_width:
            roi_draw = roi[:-(y+h-max_height), :-(x+w-max_width)]
        elif y+h > max_height:
            roi_draw = roi[:-(y+h-max_height), :]
        elif x+w > max_width:
            roi_draw = roi[:, :-(x+w-max_width)]
        img_draw[y:y+h, x:x+w] = roi_draw;
        cv2.imshow('img', img_draw)
            

cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)        

cv2.waitKey(0)
cv2.destroyAllWindows()